### Analyze latent variables for head dir, shifters, etc

In [1]:
import math
import torch
import matplotlib.pyplot as plt
import numpy as np

import torchvision

from torch.utils.data import TensorDataset, DataLoader, random_split

from utils.models import *
from utils.model_tools import *
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [2]:
torch.manual_seed(321)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

!pwd

Using cuda device
/home/mag/predictive-coding-thesis/src


### Generate latents, post prediction (at least for non-autoencoder) for all samples

In [3]:
use_headings = True
predictions = True    # if we want to get predictive latents

# I still need to work on the non-predictive because that's messed up with the headings right now
# I'd need to retrain the predictive coder model so that the displacements are not futuristic + added disp at end

transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor()
])

data_dir = 'data/noisy'

if predictions:
    if use_headings:
        dataset = HeadingDataset(source_directory=data_dir, seq_len=7, transform=transform)
    else:
        dataset = SequentialFrameDataset(source_directory=data_dir, seq_len=7
                                         transform=transform)
else:
    dataset = AutoencoderDataset(source_directory=data_dir, transform=transform)
    

print(f'{len(dataset)} samples')

batch_size = 32 if predictions else 64
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

99930 samples


In [7]:
if use_headings:
    model_file = 'faceforward_88143_samples-200ep-causal.pt'
    model = PredictiveCoderWithHead()
else:
    model_file = 'autoencoder-facenorth_72000_samples-50ep.pt'
    model = PredictiveCoder()

model.to(device)
model.load_state_dict(torch.load('model_files/' + model_file))
model.eval()
for param in model.parameters():
    param.requires_grad = False
    

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
latent_preds = []

latent_save_file = f'data/latents/predictive-{model_file[:-3]}.npy'

with torch.no_grad():
    for seq, _ in loader:
        if use_headings:
            frames, headings = seq[0].to(device), seq[1].to(device)
            latents = model.get_latent_preds(frames, headings)
        else:
            frames = seq.to(device)
            latents = model.get_latent_preds(frames)

        latent_preds.append(latents)

latent_preds = torch.stack(latent_preds, dim=0).numpy()
